Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Automated Machine Learning
**Data Preparation for Demand Forecasting Notebooks**

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data Work](#DataWork)
1. [Data Partitioning](#DataPartition)
1. [Data Upload](#DataUpload)

## 1. Introduction

The objective of this notebook is to illustrate how to pre-process the raw data and register partitioned datasets to be used in demand forecasting notebooks: (i) Demand Forecastsing Using TCN ([link placeholder]()) and (ii) Demand Forecasting Using Many Models ([link placeholder]()).

For illustration purposes we use the UCI electricity data ([link](https://archive.ics.uci.edu/ml/datasets/ElectricityLoadDiagrams20112014#)) which contains electricity consumption data for 370 consumers measured at 15 minute intervals. In this notebook we will show how to ingest the data from the original source, aggregate to an hourly frequency, select a subsample of unique time series, determine the approriate way to partition the data, and, finally, register the datasets to be used on the aforementoned notebooks.

Make sure you have executed the [configuration](https://github.com/Azure/azureml-examples/blob/b13d6f9c5cf5c042bcc265fdbc4ed741c4527ded/v1/python-sdk/configuration.ipynb) before running this notebook.

## 2. Setup

In [ ]:
import json
import logging
import os
import random

import azureml.core
import numpy as np
import pandas as pd

from azureml.core.workspace import Workspace
from matplotlib import pyplot as plt

random.seed(12345)

Accessing the Azure ML workspace requires authentication with Azure.

The default authentication is interactive authentication using the default tenant. Executing the ws = Workspace.from_config() line in the cell below will prompt for authentication the first time that it is run.

If you have multiple Azure tenants, you can specify the tenant by replacing the ws = Workspace.from_config() line in the cell below with the following:
```
from azureml.core.authentication import InteractiveLoginAuthentication
auth = InteractiveLoginAuthentication(tenant_id = 'mytenantid')
ws = Workspace.from_config(auth = auth)
```
If you need to run in an environment where interactive login is not possible, you can use Service Principal authentication by replacing the ws = Workspace.from_config() line in the cell below with the following:
```
from azureml.core.authentication import ServicePrincipalAuthentication
auth = ServicePrincipalAuthentication('mytenantid', 'myappid', 'mypassword')
ws = Workspace.from_config(auth = auth)
```
For more details, see aka.ms/aml-notebook-auth

In [ ]:
ws = Workspace.from_config()
datastore = ws.get_default_datastore()

output = {}
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

## 3. Data work
We start with declaring some of the parameters that will be used in this notebook.

- `IS_ORIGINAL_DATASET_DOWNLOADED` is a flag for wether we want to doanload the original data from the source. The flag is here to reduce the download time since the original dataset is larger than 1 GB.
- `IS_MODIFIED_DATASET_UPLOADED` is a flag for wether the datasets are uploaded to the Datastore. We use it to prevent unintentional uploads of the same datasets.
- `DOES_PARTITION_INCLUDE_VALIDATION_SET` is a placeholder for determining whether the partitioned data should include the validation set. The value True/False will be determined later in the notebook.

In [ ]:
IS_ORIGINAL_DATASET_DOWNLOADED = False
IS_MODIFIED_DATASET_UPLOADED = False
DOES_PARTITION_INCLUDE_VALIDATION_SET = (
    None  # place holder for the parameter value we will determine later.
)

Next, we specify parameters specific to the data we will work with.

- **Target column** is what we want to forecast. In our case it is electricity consumption per customer measured in kilowatt hours (kWh).
- **Time column** is the time axis along which to predict.
- **Time series identifier columns** are identified by values of the columns listed `time_series_id_column_names`. In our case all unique time series are identified by a single column `customer_id`. However, it is quite common to have multiple columns identifying unique time series. See the link for a more detailed explanation on this topic.

In [ ]:
target_column_name = "usage"
time_column_name = "date"
GRAIN_COL = "customer_id"
time_series_id_column_names = [GRAIN_COL]

In the followng block of code we will download the data from the original source to the `data` folder, load the data and print the first few rows.

In [ ]:
if not IS_ORIGINAL_DATASET_DOWNLOADED:
    print("Downloading data from the source ...\n---")
    # Download original data
    from io import BytesIO
    from urllib.request import urlopen
    from zipfile import ZipFile

    zipurl = "https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip"

    with urlopen(zipurl) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall("data")
    IS_ORIGINAL_DATASET_DOWNLOADED = True

DATA_LOCATION = os.path.join(os.getcwd(), "data")

In [ ]:
print("Printing the first few rows of the downloaded data ...\n---")
raw_df = pd.read_table("data/LD2011_2014.txt", sep=";", low_memory=False)
print(raw_df.head())

In [ ]:
raw_df.columns

The downloaded data is in what is known as a "wide" format, meaning each column name that starts with "MT_xxx" represents one unique time series. The first columnn "Unnamed: 0" is actually a time stamp at which the obervation for every time series takes place. Let's raname this column to something more meaningful. We will call it `date`.

In [ ]:
raw_df.rename(columns={raw_df.columns[0]: "date"}, inplace=True)
raw_df.head()

The "wide" data format is not usable in AutoML, which is deisgned to accept the so-called "long" format data. To tranform the data from wide to long format, we will take the each uniqe time seres (date, MT_xxx) and stack them vertically. The end result will be a data frame with 3 columns: (i) the `date` column, (ii) `custmer_id` column which is the identifier of the time series and is derived from the column name in the wide format, and (iii) `usage` which is the target varaible we are trying to model.

In [ ]:
print("Converting data from wide to long format. This may take a few minutes ...\n---")
raw_df = pd.melt(raw_df, id_vars="date", var_name="customer_id", value_name="usage")
raw_df[time_column_name] = pd.to_datetime(raw_df[time_column_name])
raw_df.to_csv("data/LD2011_2014_long_format.csv", index=False)

print("The first few rows of the data in the long format ...\n---")
raw_df.head()

In [ ]:
nseries = raw_df.groupby(time_series_id_column_names).ngroups
print("Data contains {0} individual time-series.".format(nseries))

The data tracks customers' electricity consumption every 15 minutes and is measured in kilowatt (kW) consumed. Say, the business requirement is to generate 24 hour forecast in kilowatt hours (kWh). Such forecast at 15 minute frequency results in the forecast horizon of 96 steps ahead (there 96 15-minute intervals in a 24-hour period). Moreover, if the requirement is to generate 24-hour ahead forecast, it makes sence to aggregate data measured at 15-minute intervals to an hourly frequency. This will reduce the forecast horizon by a factor of 4. The shorter the forecast horizon usually results in higher probability of achieving better forecast accuracy. 

In the next block of code we will create a `datetime` column which will identify the date and the hour of the day each observation belogs to. We also convert the target variable from kW to kWh, where $kWh = \frac{1}{4} \times kW $. After the conversion is complete, the hourly data will be stored in the `raw_hourly_df` object.

In [ ]:
raw_df.dtypes

The output of the previous command shows that the target varaible `usage` is an object. We need to transform it into a float in order to convert kW to kWh. The next command does exactly this. Because the original data contains European style format with decimals separated by commmas, we replase commas with periods before declaring target variaible as a float.

In [ ]:
raw_df[target_column_name] = (
    raw_df[target_column_name].astype(str).apply(lambda x: float(x.replace(",", ".")))
)
raw_df.dtypes

In [ ]:
# aggregate data to hourly. Here, the hourly column is called "datetime"
new_time_column_name = "datetime"

raw_df[new_time_column_name] = raw_df[time_column_name].dt.to_period("H")
raw_df[target_column_name] = raw_df[target_column_name] / 4  # convert to kWh

In [ ]:
# convert to hourly consumption by adding kWh for every 15 min interval
raw_hourly_series = raw_df.groupby([GRAIN_COL, new_time_column_name])[
    target_column_name
].sum()
raw_hourly_df = pd.DataFrame(raw_hourly_series)
raw_hourly_df.reset_index(drop=False, inplace=True)
print(raw_hourly_df.head())

del raw_df

In [ ]:
# convert time column to the datetime format
raw_hourly_df[new_time_column_name] = pd.to_datetime(
    raw_hourly_df[new_time_column_name].astype(str)
)
raw_hourly_df.to_csv(
    os.path.join(DATA_LOCATION, "hourly_data_long_format.csv"), index=False
)
raw_hourly_df.dtypes

Next, let's visualize a sample of 50 randomly selected series. The plots will be stored in the `output_folder`.

In [ ]:
all_grains = list(
    pd.unique(raw_hourly_df[time_series_id_column_names].values.ravel("K"))
)
grains_to_plot = random.sample(all_grains, k=50)
print(f"The following grains will be selected for plotting: \n{grains_to_plot}\n---")

In [ ]:
data_subset = raw_hourly_df[raw_hourly_df[GRAIN_COL].isin(grains_to_plot)]

In [ ]:
# create an output folder
OUTPUT_DIR = os.path.join(os.getcwd(), "output_folder")
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
from scripts.helper_scripts import _draw_one_plot
from matplotlib.backends.backend_pdf import PdfPages

plot_filename = "raw_ts_plots.pdf"

pdf = PdfPages(os.path.join(OUTPUT_DIR, plot_filename))
for grain, one_forecast in data_subset.groupby(GRAIN_COL):
    one_forecast[new_time_column_name] = pd.to_datetime(
        one_forecast[new_time_column_name]
    )
    one_forecast.sort_values(new_time_column_name, inplace=True)
    _draw_one_plot(
        one_forecast,
        new_time_column_name,
        target_column_name,
        time_series_id_column_names,
        pdf,
        plot_predictions=False,
    )
pdf.close()

In [ ]:
from IPython.display import IFrame

IFrame("./output_folder/raw_ts_plots.pdf", width=800, height=300)

In [ ]:
f'{os.path.join(OUTPUT_DIR, "raw_ts_plots.pdf")}'

Close examination of the consumption plots per customer shows there are quite a few customers that have no usage data prior to January 1, 2012. Some customers do not have the data until January of 2013 or 2014. It is reasonable at this point to drop all observations prior to January 1, 2012. We will call this object `clean_df`.

In [ ]:
# drop grains that have no usage as of Jan 1, 2012
tmp_df = raw_hourly_df[raw_hourly_df[new_time_column_name] == "2012-01-01 01:00:00"]
grains_to_drop = list(tmp_df[tmp_df[target_column_name] == 0][GRAIN_COL])
print(f"Number of grains to be dropped: {len(grains_to_drop)}")
del tmp_df

In [ ]:
clean_df = raw_hourly_df[~raw_hourly_df[GRAIN_COL].isin(grains_to_drop)]

# drop observations prior to 1/1/2012 since they are zero for all grains
clean_df = clean_df[clean_df[new_time_column_name] > "2011-12-31 23:00:00"]

del raw_hourly_df

To save time illustrating demand forecasting notebooks, we will use a small subset of 10 unique time series from the data.

In [ ]:
all_grains = list(pd.unique(clean_df[time_series_id_column_names].values.ravel("K")))
selected_grains = random.sample(all_grains, k=10)
print(f"The following grains will be selected:  {selected_grains}\n---")

In [ ]:
data_subset = clean_df[clean_df[GRAIN_COL].isin(selected_grains)]
nseries = data_subset.groupby(time_series_id_column_names).ngroups
print("Data subset contains {0} individual time-series.\n---".format(nseries))
data_subset.head()

In [ ]:
del all_grains, selected_grains, clean_df

In [ ]:
print("Full dataset\n---")
for grain, tmp_df in data_subset.groupby(time_series_id_column_names):
    print(
        "Grain:{}.\
    Min date: {}\
    Max date: {}\
    N: {}".format(
            grain, tmp_df["datetime"].min(), tmp_df["datetime"].max(), tmp_df.shape[0]
        )
    )
# del tmp_df, grain

In [ ]:
data_subset.to_csv(os.path.join(OUTPUT_DIR, "small_data.csv"), index=False)

## 4. Data Partitioning
The objective of this section is to determine where you want to estimate low capacity models such as many models (see [link](https://github.com/Azure/azureml-examples/tree/main/v1/python-sdk/tutorials/automl-with-azureml/forecasting-many-models)) or a deep learning model (see [link](https://github.com/Azure/azureml-examples/tree/main/v1/python-sdk/tutorials/automl-with-azureml/forecasting-github-dau) for details). Low capacity models, which include all models with the exception of TCN, require frequent retraining to maintain desired forecast accuracy. It best not to provide validation set data in such scenario. TCN, on the other hand, is a high capacity model and requires infrequent retraining. Hence, validation data is imperative here. As a result, we need to partition the data in such a way as to reflect the modelling choice.

Before deciding on the modelling framework, let's visualize the small subset of data we selected.

In [ ]:
plot_filename = "ts_plots_small_data.pdf"

pdf = PdfPages(os.path.join(OUTPUT_DIR, plot_filename))
for grain, one_forecast in data_subset.groupby(GRAIN_COL):
    one_forecast[new_time_column_name] = pd.to_datetime(
        one_forecast[new_time_column_name]
    )
    one_forecast.sort_values(new_time_column_name, inplace=True)
    _draw_one_plot(
        one_forecast,
        new_time_column_name,
        target_column_name,
        time_series_id_column_names,
        pdf,
        plot_predictions=False,
    )
pdf.close()

In [ ]:
from IPython.display import IFrame

IFrame("./output_folder/ts_plots_small_data.pdf", width=800, height=300)

One can make a determination of the modelling framework based on the visual examination of the raw time series plots. If all time series exhibit similar behavior patterns, a deep learning model can be an excellent choice in such scenario. If, on the other hand, if individual time series show heterogeneous behavior, one may be advised to run a many models accelerator which estimates one model per time seires as oppsed to one model for all time series.

In our case, one can make an argument that a deep learning model can be a good modeeling choice. As a result, we set the `DOES_PARTITION_INCLUDE_VALIDATION_SET` parameter to True. Please note, to explore the best option, you can still partition the data in such a way as to run the low capacity models. To do so, set this parametr to False.

In [ ]:
DOES_PARTITION_INCLUDE_VALIDATION_SET = True

### Generate train/valid/test/inference sets

Since deep learning models are considered "high capacity" models, they generally do not require frequent re-training. As a result, we use 2 months of data for validation and test set, respectively. The choice of 2 months is fairly arbitrary and can be modified to suit your needs. We use the 2 month of validation and test set data to reflect the infrequent re-traiing of the model given that the data frequency is hourly. Thus, there will be more than 1200 observations in the vlaidation and test sets per time series. This will give us enough data points to generate conclusions about model's performance.

This is in  constrast to the ML models that require frequent retraining and as a result require much shorter test sets to have a reasonable understanding of the model accuracy.

**Note:** Once the backtesting functionality is available, replace the statement regarding the shorter test set with the need to backtesting given a relatively frequent need to re-train the models compared to the DNNs.

In [ ]:
n_test_periods = 60 * 24
n_valid_periods = 60 * 24  # applicable only to single/TCN model
n_inference_periods = 24


def split_last_n_by_series_id(df, n, time_column_name):
    """Group df by series identifiers and split on last n rows for each group."""
    df_grouped = df.sort_values(time_column_name).groupby(  # Sort by ascending time
        time_series_id_column_names, group_keys=False
    )
    df_head = df_grouped.apply(lambda dfg: dfg.iloc[:-n])
    df_tail = df_grouped.apply(lambda dfg: dfg.iloc[-n:])
    return df_head, df_tail


train_valid_test, inference = split_last_n_by_series_id(
    data_subset, n_inference_periods, time_column_name=new_time_column_name
)

if DOES_PARTITION_INCLUDE_VALIDATION_SET:
    train_valid, test = split_last_n_by_series_id(
        train_valid_test, n_test_periods, new_time_column_name
    )
    train, valid = split_last_n_by_series_id(
        train_valid, n_valid_periods, new_time_column_name
    )
else:
    train, test = split_last_n_by_series_id(
        train_valid_test, n_test_periods, new_time_column_name
    )

We drop the target column from the inference dataset to reflect the fact that the future is unknown and the forecast is our best guess about it.

In [ ]:
inference.drop(columns=[target_column_name], inplace=True)
inference.head()

Next, we will examine the start- and end dates as well as the number of observations per time series in each of the generated datasets.

In [ ]:
print("Full dataset\n---")
for grain, tmp_df in data_subset.groupby(time_series_id_column_names):
    print(
        "Grain:{}.\
    Min date: {}\
    Max date: {}\
    N: {}".format(
            grain, tmp_df["datetime"].min(), tmp_df["datetime"].max(), tmp_df.shape[0]
        )
    )

In [ ]:
print("Train dataset\n---")
for grain, tmp_df in train.groupby(time_series_id_column_names):
    print(
        "Grain:{}.\
    Min date: {}\
    Max date: {}\
    N: {}".format(
            grain, tmp_df["datetime"].min(), tmp_df["datetime"].max(), tmp_df.shape[0]
        )
    )

In [ ]:
if DOES_PARTITION_INCLUDE_VALIDATION_SET:
    print("Valid dataset\n---")
    for grain, tmp_df in valid.groupby(time_series_id_column_names):
        print(
            "Grain:{}.\
        Min date: {}\
        Max date: {}\
        N: {}".format(
                grain,
                tmp_df["datetime"].min(),
                tmp_df["datetime"].max(),
                tmp_df.shape[0],
            )
        )

In [ ]:
print("Test dataset\n---")
for grain, tmp_df in test.groupby(time_series_id_column_names):
    print(
        "Grain:{}.\
    Min date: {}\
    Max date: {}\
    N: {}".format(
            grain, tmp_df["datetime"].min(), tmp_df["datetime"].max(), tmp_df.shape[0]
        )
    )

In [ ]:
print("Inference dataset\n---")
for grain, tmp_df in inference.groupby(time_series_id_column_names):
    print(
        "Grain:{}.\
    Min date: {}\
    Max date: {}\
    N: {}".format(
            grain, tmp_df["datetime"].min(), tmp_df["datetime"].max(), tmp_df.shape[0]
        )
    )

## 5.  Upload data to datastore
The [Machine Learning service workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-workspace), is paired with the storage account, which contains the default data store. We will use it to upload the train and test data and create [tabular datasets](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py) for training and testing. A tabular dataset defines a series of lazily-evaluated, immutable operations to load data from the data source into tabular representation.

In [ ]:
DATASET_PREFFIX_NAME = (
    "uci_electro_small_tcn"
    if DOES_PARTITION_INCLUDE_VALIDATION_SET
    else "uci_electro_small"
)
print(f"Dataset preffix name: {DATASET_PREFFIX_NAME}")

In [ ]:
IS_MODIFIED_DATASET_UPLOADED

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset

if not IS_MODIFIED_DATASET_UPLOADED:
    print("---\nUploading data ...\n---")

    train_dataset = TabularDatasetFactory.register_pandas_dataframe(
        train, target=(datastore, "dataset/"), name=f"{DATASET_PREFFIX_NAME}_train"
    )

    if DOES_PARTITION_INCLUDE_VALIDATION_SET:
        valid_dataset = TabularDatasetFactory.register_pandas_dataframe(
            valid, target=(datastore, "dataset/"), name=f"{DATASET_PREFFIX_NAME}_valid"
        )

    test_dataset = TabularDatasetFactory.register_pandas_dataframe(
        test, target=(datastore, "dataset/"), name=f"{DATASET_PREFFIX_NAME}_test"
    )

    inference_dataset = TabularDatasetFactory.register_pandas_dataframe(
        inference,
        target=(datastore, "dataset/"),
        name=f"{DATASET_PREFFIX_NAME}_inference",
    )
else:
    print("Using uploaded data ...\n---")

    target_path_train = f"{DATASET_PREFFIX_NAME}_train"
    target_path_valid = f"{DATASET_PREFFIX_NAME}_valid"
    target_path_test = f"{DATASET_PREFFIX_NAME}_test"
    target_path_inference = f"{DATASET_PREFFIX_NAME}_test"

    train_dataset = Dataset.get_by_name(ws, name=target_path_train)
    if DOES_PARTITION_INCLUDE_VALIDATION_SET:
        valid_dataset = Dataset.get_by_name(ws, name=target_path_valid)
    test_dataset = Dataset.get_by_name(ws, name=target_path_test)
    inference_dataset = Dataset.get_by_name(ws, name=target_path_inference)

### Optional

In [ ]:
# delete downladed data files to save space
import shutil

shutil.rmtree("data")